# Import timeframe data

In [2]:
import numpy as np


# Import data

In [3]:
games = np.load('/tmp/game_data.npy')
print (games.shape)

(49996, 6, 704)


# Look at data

In [4]:
#print(games[1][4])

In [5]:
labels = np.load('/tmp/game_labels.npy')
print (labels.shape)

(49996, 2)


# Separate train and test data

In [6]:

#labels_binary_length_corrected = labels.reshape((300000, 2))
labels_binary_length_corrected = labels

number_of_features_to_use = games.shape[2]

data = np.zeros((games.shape[0],games.shape[1], number_of_features_to_use))
for i in range(games.shape[0]):
    for j in range(games.shape[1]):
        data[i, j] = games[i,j][:number_of_features_to_use]
print (data.shape)


num_train_examples = int(len(data) * 0.9)
print(data.shape)
x_train = data[:num_train_examples,:]
y_train = labels_binary_length_corrected[:num_train_examples,:]
x_test = data[num_train_examples:,:]
y_test = labels_binary_length_corrected[num_train_examples:,:]

#x_train = np.reshape(x_train, x_train.shape + (1,))
#x_test = np.reshape(x_test, x_test.shape + (1,))

print(x_train.shape)

(49996, 6, 704)
(49996, 6, 704)
(44996, 6, 704)


In [ ]:
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.regularizers import l2
import keras

model = Sequential()

model.add(LSTM(
    input_shape=(None, x_train.shape[2]),
    units=800,
    return_sequences=False,
    kernel_regularizer= l2(0.01))
    )

model.add(Dense(
    units=2))
model.add(Activation('softmax'))
#model.compile(loss='mse', optimizer='rmsprop')
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.0001), metrics=[keras.metrics.mae, keras.metrics.categorical_accuracy])

In [ ]:
model_checkpoint = keras.callbacks.ModelCheckpoint('/tmp/best_weights', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
history = model.fit(x_train, y_train, epochs=40, batch_size=64, validation_split=0.2, verbose=2, callbacks = [model_checkpoint])

Train on 35996 samples, validate on 9000 samples
Epoch 1/40
12s - loss: 2.8091 - mean_absolute_error: 0.4673 - categorical_accuracy: 0.6089 - val_loss: 0.7798 - val_mean_absolute_error: 0.4484 - val_categorical_accuracy: 0.6384
Epoch 2/40
12s - loss: 0.7299 - mean_absolute_error: 0.4499 - categorical_accuracy: 0.6313 - val_loss: 0.6924 - val_mean_absolute_error: 0.4519 - val_categorical_accuracy: 0.6464
Epoch 3/40
12s - loss: 0.6799 - mean_absolute_error: 0.4465 - categorical_accuracy: 0.6379 - val_loss: 0.6667 - val_mean_absolute_error: 0.4466 - val_categorical_accuracy: 0.6438
Epoch 4/40


In [ ]:
model.load_weights('/tmp/best_weights')

In [ ]:

# Test loss and accuracy
loss_and_metrics = model.evaluate(x_test, y_test, verbose=2)
print(loss_and_metrics)

In [ ]:
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# AUC score

In [ ]:
from sklearn.metrics import roc_auc_score
y_pred = model.predict_proba(x_test, verbose=0)
score = roc_auc_score(y_test, y_pred)
print(score)

# Remove unsure predictions

In [ ]:
y_pred = model.predict_proba(x_test, verbose=0)
ind1 = y_pred[:,0] > 0.6
ind2 = y_pred[:,1] > 0.6
ind = ind1 + ind2
y_pred2 = y_pred[ind]
y_test2 = y_test[ind]
x_test2 = x_test[ind]
print(ind)
print(y_pred2.shape)
print(y_pred.shape)
score = roc_auc_score(y_test2, y_pred2)
print(score)

loss_and_metrics = model.evaluate(x_test2, y_test2, verbose=2)
print(loss_and_metrics)